# Drone Trajectory Planner

In this project, we will develop the drone trajectory planner. This notebook serves as the main file for the project, where we will refer to the instructions and demonstrate our code.

Please follow week by week instructions, which includes writing the code in the `src/` folder.

In [172]:
# Import all the files and libraries required for the project
%load_ext autoreload
%autoreload 2
import copy
    
import numpy as np

from src.camera_utils import compute_image_footprint_on_surface, compute_ground_sampling_distance, project_world_point_to_image
from src.data_model import Camera, DatasetSpec
from src.plan_computation import compute_distance_between_images, compute_speed_during_photo_capture, generate_photo_plan_on_grid
from src.visualization import plot_photo_plan

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Week 1: Introduction

No code contribution expected this week

# Week 2: Camera System Modeling and Operations

We plan to
- Model the simple pinhole camera system
- Write utility functions to
    - project a 3D world point to an image
    - Compute image footprint on a surface
    - Compute the Ground Sampling Distance

## Model the camera parameters

We want to model the following camera parameters in Python:
- focal length along x axis (in pixels)
- focal length along y axis (in pixels)
- optical center of the image along the x axis (in pixels)
- optical center of the image along the y axis (in pixels)
- Size of the sensor along the x axis (in mm)
- Size of the sensor along the y axis (in mm)
- Number of pixels in the image along the x axis
- Number of pixels in the image along the y axis

I recommend to use `dataclasses` ([Python documentation](https://docs.python.org/3/library/dataclasses.html), [Blog](https://www.dataquest.io/blog/how-to-use-python-data-classes/) to model these parameters.

$\color{red}{\text{TODO: }}$ Implement `Camera` in `src/data_model.py`

In [173]:
# Define the parameters for Skydio VT300L - Wide camera
# Ref: https://support.skydio.com/hc/en-us/articles/20866347470491-Skydio-X10-camera-and-metadata-overview
fx = 4938.56
fy = 4936.49
cx = 4095.5
cy = 3071.5
sensor_size_x_mm = 13.107 # single pixel size * number of pixels in X dimension
sensor_size_y_mm = 9.830 # single pixel size * number of pixels in Y dimension
image_size_x = 8192
image_size_y = 6144

camera_x10 = Camera(fx, fy, cx, cy, sensor_size_x_mm, sensor_size_y_mm, image_size_x, image_size_y)

In [174]:
print(f"X10 camera model: {camera_x10}")

X10 camera model: Camera(fx=4938.56, fy=4936.49, cx=4095.5, cy=3071.5, sensor_size_x_mm=13.107, sensor_size_y_mm=9.83, image_size_x_px=8192, image_size_y_px=6144)


## Project 3D world points into the image


![Camera Projection](assets/image_projection.png)
Reference: [Robert Collins CSE483](https://www.cse.psu.edu/~rtc12/CSE486/lecture12.pdf)


Equations to implement:
$$ x = f_x \frac{X}{Z} $$
$$ y = f_y \frac{Y}{Z} $$
$$ u = x + c_x $$
$$ v = y + c_y $$

$\color{red}{\text{TODO: }}$ Implement function `project_world_point_to_image` in `src/camera_utils.py`

In [175]:
point_3d = np.array([25, -30, 50], dtype=np.float32)
expected_uv = np.array([6564.80, 109.60], dtype=np.float32)
uv = project_world_point_to_image(camera_x10, point_3d)

print(f"{point_3d} projected to {uv}")

assert np.allclose(uv, expected_uv, atol=1e-2)

[ 25. -30.  50.] projected to [6564.7803   109.60571]


## Compute Image Footprint on the surface

We have written code to *project* a 3D point into the image. The reverse operation is reprojection, where we take $(x, y)$ and compute the $(X, Y)$ for a given value of $Z$. Note that while going from 3D to 2D, the depth becomes ambiguous so we need the to specify the $Z$.

An image's footprint is the area on the surface which is captured by the image. We can take the two corners of the image and reproject them at a given distance to obtain the width and length of the image.

$\color{red}{\text{TODO: }}$ Implement function `compute_image_footprint_on_surface` in `src/camera_utils.py`

In [176]:
footprint_at_100m = compute_image_footprint_on_surface(camera_x10, 100)
expected_footprint_at_100m = np.array([165.88, 124.46], dtype=np.float32)

print(f"Footprint at 100m = {footprint_at_100m}")

assert np.allclose(footprint_at_100m, expected_footprint_at_100m, atol=1e-2)


Footprint at 100m = [165.87831271 124.46090238]


In [177]:
footprint_at_200m = compute_image_footprint_on_surface(camera_x10, 200)
expected_footprint_at_200m = expected_footprint_at_100m * 2

print(f"Footprint at 200m = {footprint_at_200m}")

assert np.allclose(footprint_at_200m, expected_footprint_at_200m, atol=1e-2)

Footprint at 200m = [331.75662541 248.92180476]


## Ground Sampling Distance

Ground sampling distance is the length of the ground (in m) captured by a single pixel. We have the image footpring (the dimensions of ground captured by the whole sensor, and the number of pixels along the horizontal and vertical dimension. Can we get GSD from these two quantities?

Note: Please return just one value of the GSD. Take the mininum of the values along the two axes.

In [178]:
gsd_at_100m = compute_ground_sampling_distance(camera_x10, 100)
expected_gsd_at_100m = 0.0202

print(f"GSD at 100m: {gsd_at_100m}")

assert np.allclose(gsd_at_100m, expected_gsd_at_100m, atol=1e-4)

GSD at 100m: 0.020248817469059807


## Bonus: Reprojection from 2D to 3D

If we have a 2d pixel location of a point along with the camera model, can we go back to 3D?
Do we need any additional information.


$\color{red}{\text{TODO: }}$ Implement function `reproject_image_point_to_world` in `src/camera_utils.py` and demonstrate it by running it in the notebook. Confirm that your reprojection + projection function are consistent.

# Week 3: Model the user requirements

For this week, we will model the dataset specifications.

- Overlap: the ratio (in 0 to 1) of scene shared between two consecutive images.
- Sidelap: the ratio (in 0 to 1) of scene shared between two images in adjacent rows.
- Height: the height of the scan above the ground (in meters).
- Scan_dimension_x: the horizontal size of the rectangle to be scanned
- Scan_dimension_y: the vertical size of the rectangle to be scanned
- exposure_time_ms: the exposure time for each image (in milliseconds).


$\color{red}{\text{TODO: }}$ Implement `DatasetSpec` in `src/data_model.py`


In [179]:
# Model the nomimal dataset spec

overlap = 0.7
sidelap = 0.7
height = 30.48 # 100 ft
scan_dimension_x = 150
scan_dimension_y = 150
exposure_time_ms = 2 # 1/500 exposure time

dataset_spec = DatasetSpec(overlap, sidelap, height, scan_dimension_x, scan_dimension_y, exposure_time_ms)

print(f"Nominal specs: {dataset_spec}")

Nominal specs: DatasetSpec(overlap=0.7, sidelap=0.7, height=30.48, scan_dimension_x=150, scan_dimension_y=150, exposure_time_ms=2)


# Week 4: Compute Distance Between Photos

The overlap and sidelap are the ratio of the dimensions shared between two photos. We already know the footprint of a single image at a given distance. Can we convert the ratio into actual distances? And how does the distance on the surface relate to distance travelled by the camera?

$\color{red}{\text{TODO: }}$ Implement `compute_distance_between_images` in `src/plan_computation.py`



In [180]:
computed_distances = compute_distance_between_images(camera_x10, dataset_spec)
expected_distances = np.array([15.17, 11.38], dtype=np.float32)

print(f"Computed distance for X10 camera with nominal dataset specs: {computed_distances}")

assert np.allclose(computed_distances, expected_distances, atol=1e-2)

Computed distance for X10 camera with nominal dataset specs: [15.16791291 11.38070491]


$\color{red}{\text{TODO: }}$ define more specifications/camera parameters and check the computed distances. Does that align with your expections


Experiment 1: Increase height

- Hypothesis: The increased hight will result in the distance between images to be increased by the same amount.


In [181]:
camera_ = copy.copy(camera_x10)
dataset_spec_ = copy.copy(dataset_spec)

#Test 1: Original (100 ft) height 
computed_distances_ = compute_distance_between_images(camera_, dataset_spec_)
print(f"Computed distance at {dataset_spec_.height} meters: {computed_distances_}")

#Test 2: Distance at 200 ft
dataset_spec_.height *= 2

computed_distances_ = compute_distance_between_images(camera_, dataset_spec_)
print(f"Computed distance at {dataset_spec_.height} meters: {computed_distances_}")

#Reset height to original state
dataset_spec_.height = dataset_spec.height

#Test 3: Distance at 300 ft
dataset_spec_.height *= 3

computed_distances_ = compute_distance_between_images(camera_, dataset_spec_)
print(f"Computed distance at {dataset_spec_.height} meters: {computed_distances_}")

Computed distance at 30.48 meters: [15.16791291 11.38070491]
Computed distance at 60.96 meters: [30.33582583 22.76140983]
Computed distance at 91.44 meters: [45.50373874 34.14211474]


Experiment 2: Increase Focal length

- Hypothesis: The increased focal length will result in a zoomed in photo which will decrease the result of the distance 


In [182]:
camera_ = copy.copy(camera_x10)
dataset_spec_ = copy.copy(dataset_spec)

#Test 1: Original Calculation
computed_distances_ = compute_distance_between_images(camera_, dataset_spec_)
print(f"Computed distance: {computed_distances_}")

#Test 2: Test at 1.5x 
camera_.fx *= 1.5
camera_.fy *= 1.5

computed_distances_ = compute_distance_between_images(camera_, dataset_spec_)
print(f"Computed distance: {computed_distances_}")

#Reset fx and fy to original state
camera_.fx = camera_x10.fx
camera_.fy = camera_x10.fy

#Test 3: Test at 3x 
camera_.fx *= 3
camera_.fy *= 3

computed_distances_ = compute_distance_between_images(camera_, dataset_spec_)
print(f"Computed distance: {computed_distances_}")

Computed distance: [15.16791291 11.38070491]
Computed distance: [10.11194194  7.58713661]
Computed distance: [5.05597097 3.7935683 ]


## Bonus: Non-Nadir photos

We have solved for the distance assuming that the camera is facing straight down to the ground. This is called [Nadir scanning](https://support.esri.com/en-us/gis-dictionary/nadir). However, in practise we might want a custom gimbal angle.

Your bonus task is to make the distance computation general. Introduce a double `camera_angle` parameter (which is the angle from the X-axis) in the dataset specification, and work out how to adapt your computation. Feel free to reach out to Ayush to discuss ideas and assumptions!

![Non Nadir Footprint](assets/non_nadir_gimbal_angle.png)

# Week 5: Compute Maximum Speed For Blur Free Photos

To restrict motion blur due to camera movement to tolerable limits, we need to restrict the speed such that the image contents move less than 1px away. 

How much does 1px of movement translate to movement of the scene on the ground? It is the ground sampling distance!
From previous week, we know that this is the maximum movement the camera can have. 
We have the distance now. To get speed we need to divide it with time. Do we have time already in our data models?

$\color{red}{\text{TODO: }}$ Implement `compute_speed_during_photo_capture` in `src/plan_computation.py`.

In [183]:
computed_speed = compute_speed_during_photo_capture(camera_x10, dataset_spec, allowed_movement_px=1)
expected_speed = 3.09

print(f"Computed speed during photo captures: {computed_speed:.2f}")

assert np.allclose(computed_speed, expected_speed, atol=1e-2)

Computed speed during photo captures: 3.09


$\color{red}{\text{TODO: }}$ define more specifications/camera parameters and check the computed distances. Does that align with your expections


Experiment 1: Increasing the allowed movement pixels

- Hypothesis: This will result in an increase in the computed speed. The allowed movement pixels will add motion blur to the photos, caused by the drone going at a faster speed than the exposure time

In [184]:
camera_ = copy.copy(camera_x10)
dataset_spec_ = copy.copy(dataset_spec)

#Test 1: Default Allowed Pixels
computed_speed_ = compute_speed_during_photo_capture(camera_, dataset_spec_)
print(f"Computed speed: {computed_speed_:.2f} m/s")

#Test 2: 1.5 Allowed Pixels 
computed_speed_ = compute_speed_during_photo_capture(camera_, dataset_spec_, 1.5)
print(f"Computed speed: {computed_speed_:.2f} m/s")

#Test 3: 2 Allowed Pixels
computed_speed_ = compute_speed_during_photo_capture(camera_, dataset_spec_, 2)
print(f"Computed speed: {computed_speed_:.2f} m/s")

#Bonus Test: 1/2 Allowed Pixel
#This will decrease the computed speed since the images will be sharper, caused by a slower speed
computed_speed_ = compute_speed_during_photo_capture(camera_, dataset_spec_, 0.5)
print(f"Computed speed: {computed_speed_:.2f} m/s")

Computed speed: 3.09 m/s
Computed speed: 4.63 m/s
Computed speed: 6.17 m/s
Computed speed: 1.54 m/s


Experiment 2: Increasing the distance from the surface 

- Hypothesis: This will result in an increase in the computed speed. The increased height will result in a increased calculation by the GSD function. This will result in a bigger ratio in the GSD/Time function done in calculating speed

In [185]:
camera_ = copy.copy(camera_x10)
dataset_spec_ = copy.copy(dataset_spec)
original_height = dataset_spec_.height
 
#Test 1: Default (100 ft) Height
computed_speed_ = compute_speed_during_photo_capture(camera_, dataset_spec_)
print(f"Computed speed: {computed_speed_:.2f} m/s")

#Test 2: 200 ft 
dataset_spec_.height * 2
computed_speed_ = compute_speed_during_photo_capture(camera_, dataset_spec_, 1.5)
print(f"Computed speed: {computed_speed_:.2f} m/s")

#Reset height
dataset_spec_.height = original_height

#Test 3: 300 ft
dataset_spec_.height * 3
computed_speed_ = compute_speed_during_photo_capture(camera_, dataset_spec_, 2)
print(f"Computed speed: {computed_speed_:.2f} m/s")


Computed speed: 3.09 m/s
Computed speed: 4.63 m/s
Computed speed: 6.17 m/s


# Week 6: Generate Full Flight Plans  

We now have all the tools to generate the full flight plan.

Steps for this week:
1. Define the `Waypoint` data model. What attributes should the data model have?
   1. For Nadir scans, just the position of the camera is enough as we will always look drown to the ground.
   2. For general case (bonus), we also need to define where the drone will look at.
3. Implement the function `generate_photo_plan_on_grid` to generate the full plan.
   1. Compute the maximum distance between two images, horizontally and vertically.
   2. Layer the images such that we cover the whole scan area. Note that you need to take care when the scan dimension is not a multiple of distance between images. Example: to cover 45m length with 10m between images, we would need 4.5 images. Not possible. 4 images would not satisfy the overlap, so we should go with 5. How should we arrange 5 images in the given 45m.
   3. Assign the speed to each waypoint.

$\color{red}{\text{TODO: }}$ Implement:
- `Waypoint` in `src/data_model.py`
- `generate_photo_plan_on_grid` in `src/plan_computation.py`.

In [186]:
computed_plan = generate_photo_plan_on_grid(camera_x10, dataset_spec) 

print(f"Computed plan with {len(computed_plan)} waypoints")

Computed plan with 165 waypoints


In [187]:
MAX_NUM_WAYPOINTS_TO_PRINT = 20

for idx, waypoint in enumerate(computed_plan[:20]):
    print(f"Idx {idx}: {waypoint}")
if len(computed_plan) >= MAX_NUM_WAYPOINTS_TO_PRINT:
    print("...")

Idx 0: Waypoint(x=0.0, y=0.0, z=30.48, speed=np.float64(3.0859197822847144))
Idx 1: Waypoint(x=15.0, y=0.0, z=30.48, speed=np.float64(3.0859197822847144))
Idx 2: Waypoint(x=30.0, y=0.0, z=30.48, speed=np.float64(3.0859197822847144))
Idx 3: Waypoint(x=45.0, y=0.0, z=30.48, speed=np.float64(3.0859197822847144))
Idx 4: Waypoint(x=60.0, y=0.0, z=30.48, speed=np.float64(3.0859197822847144))
Idx 5: Waypoint(x=75.0, y=0.0, z=30.48, speed=np.float64(3.0859197822847144))
Idx 6: Waypoint(x=90.0, y=0.0, z=30.48, speed=np.float64(3.0859197822847144))
Idx 7: Waypoint(x=105.0, y=0.0, z=30.48, speed=np.float64(3.0859197822847144))
Idx 8: Waypoint(x=120.0, y=0.0, z=30.48, speed=np.float64(3.0859197822847144))
Idx 9: Waypoint(x=135.0, y=0.0, z=30.48, speed=np.float64(3.0859197822847144))
Idx 10: Waypoint(x=150.0, y=0.0, z=30.48, speed=np.float64(3.0859197822847144))
Idx 11: Waypoint(x=150.0, y=10.714285714285714, z=30.48, speed=np.float64(3.0859197822847144))
Idx 12: Waypoint(x=135.0, y=10.714285714285

## Bonus: Time computation 

if you have some time, you can implement a time computation function. We can make the drone fly as fast as possible between photos, but make sure it can decelerate back to the required speed at the photos. Please use the following data: 
- Max drone speed: 16m/s.
- Max acceleration: 3.5 m/s^2.

Hint: you might need to use a trapezoidal speed profile

# Week 7: Visualize Flight Plans

This week, we will use a third party plotting framework called [Plotly](https://plotly.com/python/) to visualize our plans. Please follow this [tutorial](https://www.kaggle.com/code/kanncaa1/plotly-tutorial-for-beginners) to gain some basic experience with Plotly, and then come up with your own visualization function. You are free to choose to come up with your own visualization, and use something other than Plotly.

$\color{red}{\text{TODO: }}$ Implement `plot_photo_plan` in `src/visualization.py`

In [188]:
fig = plot_photo_plan(computed_plan)
fig.show()

$\color{red}{\text{TODO: }}$ Compute the following ablations (and any other you can think of). 
You need to describe the input params you are changing, what impact you can observe, explanation behind the change in output, and practical implication of the correlation.

1. Change overlap and confirm it affects the consecutive images

In [189]:
camera_ = copy.deepcopy(camera_x10)
dataset_spec_ = copy.deepcopy(dataset_spec)

dataset_spec_.exposure_time_ms = 1000
dataset_spec_.overlap = 0.2

print(camera_, dataset_spec_)
fig = plot_photo_plan(generate_photo_plan_on_grid(camera_, dataset_spec_))
fig.show()

Camera(fx=4938.56, fy=4936.49, cx=4095.5, cy=3071.5, sensor_size_x_mm=13.107, sensor_size_y_mm=9.83, image_size_x_px=8192, image_size_y_px=6144) DatasetSpec(overlap=0.2, sidelap=0.7, height=30.48, scan_dimension_x=150, scan_dimension_y=150, exposure_time_ms=1000)


Observation: Changing the overlap, in this case decreasing it, will result in a decrease in waypoints. It is confirmed that the overlap will have an affect on the output.

Explanation: The overlap results in the spacing between the photos to change on the x axis. A higher overlap requires more photos to cover the same area as compared to a lower overlap. 

Practical Implication: A smaller overlap will result in fewer waypoints that results in faster times due to fewer photos being taken. Subsequently, taking fewer photos can also reduce the storage taken by the photos, or allow for higher quality photos on restricted storage

2. Change sidelap and confirm it does not affect the consecutive images

In [190]:
camera_ = copy.deepcopy(camera_x10)
dataset_spec_ = copy.deepcopy(dataset_spec)

dataset_spec_.exposure_time_ms = 1000
dataset_spec_.sidelap = 0.2

print(camera_, dataset_spec_)
fig = plot_photo_plan(generate_photo_plan_on_grid(camera_, dataset_spec_))
fig.show()

Camera(fx=4938.56, fy=4936.49, cx=4095.5, cy=3071.5, sensor_size_x_mm=13.107, sensor_size_y_mm=9.83, image_size_x_px=8192, image_size_y_px=6144) DatasetSpec(overlap=0.7, sidelap=0.2, height=30.48, scan_dimension_x=150, scan_dimension_y=150, exposure_time_ms=1000)


Observation: Changing the sidelap, in this case decreasing it, will result in a decrease in waypoints. It is debunked that the sidelap will not have an affect on the output.

Explanation: The sidelap results in the spacing between the photos to change on the y axis. A higher sidelap requires more photos to cover the same area as compared to a lower sidelap. 

Practical Implication: A smaller sidelap will result in fewer waypoints that results in faster times due to fewer photos being taken. Subsequently, taking fewer photos can also reduce the storage taken by the photos, or allow for higher quality photos on restricted storage

3. Change the height of the scan and document the affect on scan plans

In [191]:
camera_ = copy.deepcopy(camera_x10)
dataset_spec_ = copy.deepcopy(dataset_spec)

dataset_spec_.exposure_time_ms = 1000
dataset_spec_.height *= 1.5

print(camera_, dataset_spec_)
fig = plot_photo_plan(generate_photo_plan_on_grid(camera_, dataset_spec_))
fig.show()

Camera(fx=4938.56, fy=4936.49, cx=4095.5, cy=3071.5, sensor_size_x_mm=13.107, sensor_size_y_mm=9.83, image_size_x_px=8192, image_size_y_px=6144) DatasetSpec(overlap=0.7, sidelap=0.7, height=45.72, scan_dimension_x=150, scan_dimension_y=150, exposure_time_ms=1000)


Observation: Changing the height, in this case increasing it, will result in a decrease in waypoints. This affects both the x and y axis compared to just chaging overlap or sidelap respectively. It is confirmed that the height will not have an affect on the output.

Explanation: The higher height of the drone will result in a bigger area being captured. Since the area being captures is bigger, the drone will have to move farther to capture the next area. 

Practical Implication: A higher height will require less photos to be taken. This has the same benefits of quicker time and storage savings. A disadvantage will be the loss of detail quality due to the farther distance from the subject (the ground) and the camera. 

4. Change exposure time


In [192]:
camera_ = copy.deepcopy(camera_x10)
dataset_spec_ = copy.deepcopy(dataset_spec)
photo_plan = generate_photo_plan_on_grid(camera_, dataset_spec_)

print("Total exposure time for original flight plan:",(dataset_spec.exposure_time_ms * len(photo_plan))/1000, "seconds")
dataset_spec_.exposure_time_ms = 500
print("Total exposure time for modified flight plan:",(dataset_spec_.exposure_time_ms * len(photo_plan))/1000, "seconds")

print(camera_, dataset_spec_)
fig = plot_photo_plan(generate_photo_plan_on_grid(camera_, dataset_spec_))
fig.show()

Total exposure time for original flight plan: 0.33 seconds
Total exposure time for modified flight plan: 82.5 seconds
Camera(fx=4938.56, fy=4936.49, cx=4095.5, cy=3071.5, sensor_size_x_mm=13.107, sensor_size_y_mm=9.83, image_size_x_px=8192, image_size_y_px=6144) DatasetSpec(overlap=0.7, sidelap=0.7, height=30.48, scan_dimension_x=150, scan_dimension_y=150, exposure_time_ms=500)


Observation: Changing the exposure time will not have a direct change on the waypoints required for the plotting, but has an effect on the total time for the flight plan.

Explanation: The longer exposure time will result in a longer time required for the flight plan. 

Practical Implication: A longer exposure time will be beneficial during runs that are done in the night time or cloudy conditions. The higher exposure will result in brighter photos and an increase in detail for non optimal conditions. Alternatively, a shorter exposure time can save time when needed.
